<a href="https://colab.research.google.com/github/olgaki/chatbot_examples/blob/main/chatbot_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!streamlit run lstm-chatbot.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.16.151.254
⠙⠹⠸⠼⠴your url is: https://social-cobras-repeat.loca.lt


In [ ]:
import streamlit as st
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# ------------------------------
# Environment to manage state
# ------------------------------
class RestaurantEnv:
    def __init__(self):
        self.slots = {"people": None, "day": None, "time": None}

    def reset(self):
        for k in self.slots:
            self.slots[k] = None

    def get_state_vector(self):
        return np.array([float(v is not None) for v in self.slots.values()])

    def is_booking_ready(self):
        return all(self.slots.values())

    def fill_slot(self, slot, value):
        if slot in self.slots:
            self.slots[slot] = value

# ------------------------------
# Create and train LSTM model
# ------------------------------
def create_model(input_dim=3, output_dim=4):
    model = Sequential()
    model.add(LSTM(64, input_shape=(1, input_dim), return_sequences=True))
    model.add(LSTM(32))
    model.add(Dense(output_dim, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model

def train_lstm_on_synthetic_data(model):
    X = []
    y = []

    training_samples = [
        ([0, 0, 0], 0),  # Ask for people
        ([1, 0, 0], 1),  # Ask for day
        ([1, 1, 0], 2),  # Ask for time
        ([1, 1, 1], 3),  # Confirm
    ]

    for state, action in training_samples:
        for _ in range(100):
            noisy = [min(1.0, max(0.0, s + random.uniform(-0.1, 0.1))) for s in state]
            X.append(noisy)
            y.append(action)

    X = np.array(X).reshape(-1, 1, 3)
    y = np.eye(4)[y]

    model.fit(X, y, epochs=50, verbose=0)
    return model

# ------------------------------
# Initialize session state
# ------------------------------
if "env" not in st.session_state:
    st.session_state.env = RestaurantEnv()
    st.session_state.env.reset()

    model = create_model()
    trained_model = train_lstm_on_synthetic_data(model)
    st.session_state.model = trained_model

    st.session_state.last_slot_asked = None
    st.session_state.messages = [{"role": "assistant", "content": "Hi! I can help you book a table. Let's get started!"}]

# ------------------------------
# Show chat history
# ------------------------------
for msg in st.session_state.messages:
    with st.chat_message(msg["role"]):
        st.markdown(msg["content"])

# ------------------------------
# Handle user input
# ------------------------------
if prompt := st.chat_input("Type your message..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    env = st.session_state.env

    # Fill previously requested slot
    if st.session_state.last_slot_asked:
        env.fill_slot(st.session_state.last_slot_asked, prompt)

    # Predict next action
    state_vec = env.get_state_vector().reshape(1, 1, 3)
    prediction = st.session_state.model.predict(state_vec, verbose=0)
    action = int(np.argmax(prediction))

    # Define action meanings
    slot_map = {0: "people", 1: "day", 2: "time"}
    slot_prompts = {
        "people": "How many people should I book for?",
        "day": "Which day would you like the booking?",
        "time": "What time should I book it for?"
    }

    if action == 3 and env.is_booking_ready():
        response = f"✅ Your table is booked for {env.slots['people']} people on {env.slots['day']} at {env.slots['time']}! 🎉"
        st.session_state.last_slot_asked = None
    else:
        slot = slot_map.get(action, None)
        response = slot_prompts.get(slot, "Could you clarify something?")
        st.session_state.last_slot_asked = slot

    st.session_state.messages.append({"role": "assistant", "content": response})
    st.rerun()


In [6]:
! pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 117.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.4 MB/s eta 0:00:00


In [11]:
!lsof -i :8501  # Kill if occupied
!pkill -f streamlit

In [1]:
!pip install streamlit==1.28.0 tensorflow==2.15.0

In [3]:
!npm install localtunnel


y
y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸
added 22 packages in 3s
⠸
⠸3 packages are looking for funding
⠸  run `npm fund` for details
⠸